In [769]:
import openai
import json
import os
import time

In [770]:
openai.api_key = ''

In [771]:
PROMPTS = [
    """
        患者想了解{name}。请根据疾病简介：{desc}，在科室场景：{cure_department}，生成仅一个患者与专业医师之间的问答对，其中应包括多样化的场景、背景或身份信息以丰富对话内容。
        对话应包括患者关于{name}是什么病的咨询，专业医师对{name}进行介绍。回答内容避免使用第一人称主语。
    """,
    """
        患者想了解{name}的病因。请根据疾病病因：{cause}，在科室场景：{cure_department}，生成仅一个患者与专业医师之间的问答对，其中应包括多样化的场景、背景或身份信息以丰富对话内容。
        对话应包括患者关于{name}的病因的咨询，专业医师进行解答。回答内容避免使用第一人称主语。
    """,
    """
        患者想了解如何预防{name}。请根据以下预防措施：{prevent}，在科室场景：{cure_department}，生成仅一个患者与专业医师之间的问答对，其中应包括多样化的场景、背景或身份信息以丰富对话内容。
        对话应包括患者关于{name}的预防措施的咨询，专业医师进行解答。回答内容避免使用第一人称主语。
    """,
    """
        患者描述了自己感受到的症状或者去医院检查出来的症状：{symptom}，这些症状指向了疾病名称：{name}。请根据以上内容，在科室场景：{cure_department}，生成仅一个患者与专业医师之间的问答对，其中应包括多样化的场景、背景或身份信息以丰富对话内容。
        对话应包括患者描述症状，并询问自己是否得了什么病。注意普通的症状需要描述为患者自己感受到的，复杂的症状需要描述为去医院检查出来的。专业医师诊断所患疾病为{name}。回答内容避免使用第一人称主语。
    """,
    """
        患者想了解{name}可能存在的症状。请根据疾病症状：{symptom}，在科室场景：{cure_department}，生成仅一个患者与专业医师之间的问答对，其中应包括多样化的场景、背景或身份信息以丰富对话内容。
        对话应包括患者关于{name}会有哪些症状的咨询，专业医师进行解答。回答内容避免使用第一人称主语。
    """,
    """
        患者想了解{name}的诊断需要做哪些检查项目。请根据检查项目：{check}，在科室场景：{cure_department}，生成仅一个患者与专业医师之间的问答对，其中应包括多样化的场景、背景或身份信息以丰富对话内容。
        对话应包括患者关于{name}的诊断需要做哪些检查项目的咨询，专业医师列出所需检查项目。回答内容避免使用第一人称主语。
    """,
    """
        患者想了解{name}有什么并发疾病。请根据并发疾病：{acompany}，在科室场景：{cure_department}，生成仅一个患者与专业医师之间的问答对，其中应包括多样化的场景、背景或身份信息以丰富对话内容。
        对话应包括患者关于{name}有什么并发疾病的咨询，专业医师进行解答。回答内容避免使用第一人称主语。
    """,
    """
        患者想了解{name}的风险因素。请根据患病途径：{get_way}；患病率：{get_prob}；易感人群：{easy_get}，在科室场景：{cure_department}，生成仅一个患者与专业医师之间的问答对，其中应包括多样化的场景、背景或身份信息以丰富对话内容。
        患者的问题应包括{name}的患病途径、患病率和易感人群的咨询。问题包含多个提问时应在一个问题中全部问完。
        专业医师的回答中，{name}的患病途径、患病率和易感人群都需给出。回答内容避免使用第一人称主语。
    """,
    """
        患者想了解{name}的治疗信息。请根据治疗方式：{cure_way}；治愈率：{cured_prob}；治疗周期：{cure_lasttime}，在科室场景：{cure_department}，生成仅一个患者与专业医师之间的问答对，其中应包括多样化的场景、背景或身份信息以丰富对话内容。
        患者的问题应包括{name}的治疗方式、治愈率和治疗周期的咨询。问题包含多个提问时应在一个问题中全部问完。
        专业医师的回答中，{name}的治疗方式、治愈率和治疗周期都需给出。回答内容避免使用第一人称主语。
    """,
    """
        患者想了解{name}的费用信息。请根据治疗费用：{cost_money}；是否医保：{yibao_status}，在科室场景：{cure_department}，生成仅一个患者与专业医师之间的问答对，其中应包括多样化的场景、背景或身份信息以丰富对话内容。
        患者的问题应包括{name}的治疗费用和该疾病的治疗是否有医保的咨询。问题包含多个提问时应在一个问题中全部问完。
        专业医师的回答中，{name}的治疗费用和医保状态都需给出。回答内容避免使用第一人称主语。
    """,
    """
        患者想了解确诊了{name}该吃什么药。请根据常用药品：{common_drug}；推荐药品：{recommand_drug}，在科室场景：{cure_department}，生成仅一个患者与专业医师之间的问答对，其中应包括多样化的场景、背景或身份信息以丰富对话内容。
        对话应包括患者关于{name}该吃什么药的咨询，专业医师进行解答。回答内容避免使用第一人称主语。
    """,
    """
        患者已确诊{name}，想了解某个药品的具体品牌名称。请根据推荐药品：{recommand_drug}；药品具体品牌名称：{drug_detail}，在科室场景：{cure_department}，生成仅一个患者与专业医师之间的问答对，其中应包括多样化的场景、背景或身份信息以丰富对话内容。
        对话应包括患者关于推荐药品中某个药品的具体品牌、名字叫什么的咨询，专业医师给出药品具体品牌名称中对应于该药品的几个具体名字。回答内容避免使用第一人称主语。
    """,
    """
        患者已确诊{name}，想了解在饮食方面的注意事项。请根据宜吃食物：{do_eat}；忌吃食物：{not_eat}，在科室场景：{cure_department}，生成仅一个患者与专业医师之间的问答对，其中应包括多样化的场景、背景或身份信息以丰富对话内容。
        患者的问题应包括关于{name}患者能吃什么不能吃什么的咨询。问题包含多个提问时应在一个问题中全部问完。
        专业医师的回答中，{name}的宜吃食物和忌吃食物都需给出。回答内容避免使用第一人称主语。
    """,
    """
        患者已确诊{name}，想了解食谱推荐。请根据推荐食谱：{recommand_eat}，在科室场景：{cure_department}，生成仅一个患者与专业医师之间的问答对，其中应包括多样化的场景、背景或身份信息以丰富对话内容。
        对话应包括患者关于{name}患者食谱推荐的咨询，专业医师进行解答。回答内容避免使用第一人称主语。
    """
]

In [772]:
def read_oncology_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

In [773]:
def generate_qa(instance, prompts):
    name = instance.get('name','')
    cure_department = instance.get('cure_department','')
    desc = instance.get('desc','')
    cause = instance.get('cause','')
    prevent = instance.get('prevent','')
    symptom = instance.get('symptom','')
    check = instance.get('check','')
    acompany = instance.get('acompany','')
    get_way = instance.get('get_way','')
    get_prob = instance.get('get_prob','')
    easy_get = instance.get('easy_get','')
    cure_way = instance.get('cure_way','')
    cured_prob = instance.get('cured_prob','')
    cure_lasttime = instance.get('cure_lasttime','')
    cost_money = instance.get('cost_money','')
    yibao_status = instance.get('yibao_status','')
    common_drug = instance.get('common_drug','')
    recommand_drug = instance.get('recommand_drug','')
    drug_detail = instance.get('drug_detail','')
    do_eat = instance.get('do_eat','')
    not_eat = instance.get('not_eat','')
    recommand_eat = instance.get('recommand_eat','')

    responses = []

    for PROMPT in PROMPTS:
        if PROMPT == PROMPTS[0]:
            if all(context in ['', []] for context in [desc]):
                continue
        elif PROMPT == PROMPTS[1]:
            if all(context in ['', []] for context in [cause]):
                continue
        elif PROMPT == PROMPTS[2]:
            if all(context in ['', []] for context in [prevent]):
                continue
        elif PROMPT == PROMPTS[3]:
            if all(context in ['', []] for context in [symptom]):
                continue
        elif PROMPT == PROMPTS[4]:
            if all(context in ['', []] for context in [symptom]):
                continue
        elif PROMPT == PROMPTS[5]:
            if all(context in ['', []] for context in [check]):
                continue
        elif PROMPT == PROMPTS[6]:
            if all(context in ['', []] for context in [acompany]):
                continue
        elif PROMPT == PROMPTS[7]:
            if all(context in ['', []] for context in [get_way, get_prob, easy_get]):
                continue
        elif PROMPT == PROMPTS[8]:
            if all(context in ['', []] for context in [cure_way, cured_prob, cure_lasttime]):
                continue
        elif PROMPT == PROMPTS[9]:
            if all(context in ['', []] for context in [cost_money, yibao_status]):
                continue
        elif PROMPT == PROMPTS[10]:
            if all(context in ['', []] for context in [common_drug, recommand_drug]):
                continue
        elif PROMPT == PROMPTS[11]:
            if all(context in ['', []] for context in [recommand_drug, drug_detail]):
                continue
        elif PROMPT == PROMPTS[12]:
            if all(context in ['', []] for context in [do_eat, not_eat]):
                continue
        elif PROMPT == PROMPTS[13]:
            if all(context in ['', []] for context in [recommand_eat]):
                continue

        prompt = PROMPT.format(
            name=name,
            cure_department=cure_department,
            desc=desc,
            cause=cause,
            prevent=prevent,
            symptom=symptom,
            check=check,
            acompany=acompany,
            get_way=get_way,
            get_prob=get_prob,
            easy_get=easy_get,
            cure_way=cure_way,
            cured_prob=cured_prob,
            cure_lasttime=cure_lasttime,
            cost_money=cost_money,
            yibao_status=yibao_status,
            common_drug=common_drug,
            recommand_drug=recommand_drug,
            drug_detail=drug_detail,
            do_eat=do_eat,
            not_eat=not_eat,
            recommand_eat=recommand_eat
        )
        prompt += """
                请直接按以下json文件格式返回，input空着，不要有任何其他内容：
                {
                    "instruction": "患者自己提问的内容",
                    "input": "",
                    "output": "专业医师回答的内容"
                }
            """

        #time.sleep(9)

        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "你是一位医疗专家。"},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=1024,
                n=1,
                stop=None,
                temperature=0.7
            )
        
            generated_text = response['choices'][0]['message']['content'].strip()
            #print(generated_text)
            
            generated_json = json.loads(generated_text)
            responses.append(generated_json)

        except Exception as e:
            print(f"生成失败: {e}")
            responses.append("生成失败")
    return responses

In [774]:
def append_to_json_file(responses, output_file):
    if not os.path.exists(output_file):
        with open(output_file, 'w', encoding='utf-8') as file:
            data = []
            for response in responses:
                data.append(response)
            json.dump(data, file, ensure_ascii=False, indent=4)
    else:
        with open(output_file, 'r+', encoding='utf-8') as file:
            current_data = json.load(file)
            for response in responses:
                current_data.append(response)
            file.seek(0)
            json.dump(current_data, file, ensure_ascii=False, indent=4)

In [775]:
def process_json_instances(input_file, output_file):
    data = read_oncology_data(input_file)

    for instance in data:
        responses = generate_qa(instance, PROMPTS)
        append_to_json_file(responses, output_file)

*102681+11409=114090*

In [ ]:
input_file_path = 'dataset/CMKG/medical_data.json'
output_file_path = 'dataset/cMKGQA/data.json'
process_json_instances(input_file_path, output_file_path)